# Parameter Estimation: Example 1

## Estimation under square loss

To ease the estimation process when given data, a separate module
{mod}`ode_loss` has been constructed for observations coming from a single
state. We demonstrate how to do parametre fitting using two models; first, a the
SIR model, followed by the Legrand SEIHFR model from {citets}`Legrand2007` [\[Legrand2007\]]() used 
for Ebola in the next page {doc}`.estimate2`.

### SIR Model

We set up an SIR model as seen previously in {doc}`.sir`.

In [ ]:
from pygom import SquareLoss, common_models

import numpy

import scipy.integrate

import matplotlib.pyplot

# define the parameters
paramEval = [('beta',0.5), ('gamma',1.0/3.0)]

Initialize the model using the preloaded common model {obj}`.SIR`.

In [ ]:
ode = common_models.SIR(paramEval)

We assume that we have perfect information about the $R$
compartment.

In [ ]:
x0 = [1, 1.27e-6, 0]

# time, including the initial time t0 at t=0

t = numpy.linspace(0, 150, 1000)

# determine the solution.

solution = scipy.integrate.odeint(ode.ode, x0, t)

#TODO why use scipy?

y = solution[:,1:3].copy()


Initialize the class with some initial guess


# our initial guess

theta = [0.2, 0.2]

objSIR = SquareLoss(theta, ode, x0, t[0], t[1::], y[1::,:], ['I','R'])



Note that we need to provide the initial values, $x_{0}$ and $t_{0}$
differently to the observations $y$ and the corresponding time $t$.
Additionally, the state which the observation lines needs to be
specified. Either a single state, or multiple states are allowed, as
seen above.



### Difference in gradient

We have provided two different ways of obtaining the gradient, these are
explained in {doc}`.gradient` in a bit more detail. First, lets see how
similar the outputs of the two methods are.

In [ ]:

objSIR.sensitivity()

objSIR.adjoint()



and the time required to obtain the gradient for the SIR model under
$\theta = (0.2,0.2)$, previously entered.



%timeit objSIR.sensitivity()


%timeit objSIR.adjoint()

The amount of time taken for both method is dependent on the
number of observations as well as the number of states. The effect on
the adjoint method as the number of observations differs can be quite
evident. This is because the adjoint method is under a discretization
which loops in Python where as the forward sensitivity equations are
solved via an integration. As the number of observation gets
larger, the affect of the Python loop becomes more obvious.

The difference in gradient is larger when there are less observations. This
is because the adjoint method use interpolations on the output of the
ode between each consecutive time points. Given solutions over the same
length of time, fewer discretizations leads to a less accurate
interpolation. Note that the interpolation is currently performed using a
univariate spline, due to the limitation of Python packages. Ideally,
one would prefer to use an (adaptive) Hermite or Chebyshev
interpolation. 

#TODO add refs

Note how we ran the two gradient functions once before
timing it, that is because we only find the properties (Jacobian,
gradient) of the ODEs during runtime.



### Optimized result

Then standard optimization procedures with some suitable initial guess
should yield the correct result. It is important to set the boundaries
for compartmental models as we know that all the parameters are strictly
positive. We put a less restrictive inequality here for demonstration
purpose.


In [ ]:
# what we think the bounds are

boxBounds = [(0.0,2.0),(0.0,2.0)]

Then using the optimization routines in `scipy.optimize`, for example,
the *SLSQP* method with the gradient obtained by forward sensitivity.



In [ ]:
from scipy.optimize import minimize

res = minimize(fun=objSIR.cost, jac=objSIR.sensitivity, x0=theta, 
               bounds=boxBounds, method='SLSQP')

print(res)


Other methods available in `scipy.optimize.minimize` can also be used,
such as the *L-BFGS-B* and *TNC*. We can also use methods that accepts
the exact Hessian such as *trust-ncg* but that should not be necessary
most of the time.

#TODO add doc refs for scipy